In [1]:
# Librerías
import pypdb
import pandas as pd
import numpy as np
import csv
import pickle
from prody import *
from Bio import pairwise2, SeqIO
import sys, os
# Añadimos la ruta del directorio raíz para tener acceso a los archivos y módulos
sys.path.append(r'..')

In [2]:
# Nombre de la proteina de interes
prot_name = 'fxa'
# Secuencia P28482 (ERK2_HUMAN)
uniprot_id = "P00742"

In [3]:
# Secuencia de la CDK2 de UniProt
import requests
from Bio import SeqIO

# Descargamos el fasta de UniProt
url_fasta = requests.get("https://www.uniprot.org/uniprot/" + uniprot_id + ".fasta")
file_name_fasta = '../data/' + uniprot_id + '.fasta'
open(file_name_fasta, 'wb').write(url_fasta.content)
# Leemos la secuenciade aminoácidos
fasta_prot = SeqIO.read(open(file_name_fasta),'fasta')
seq_prot = str(fasta_prot.seq)
print(seq_prot)
print(F'\nSequence length: {len(seq_prot)} aa.')

MGRPLHLVLLSASLAGLLLLGESLFIRREQANNILARVTRANSFLEEMKKGHLERECMEETCSYEEAREVFEDSDKTNEFWNKYKDGDQCETSPCQNQGKCKDGLGEYTCTCLEGFEGKNCELFTRKLCSLDNGDCDQFCHEEQNSVVCSCARGYTLADNGKACIPTGPYPCGKQTLERRKRSVAQATSSSGEAPDSITWKPYDAADLDPTENPFDLLDFNQTQPERGDNNLTRIVGGQECKDGECPWQALLINEENEGFCGGTILSEFYILTAAHCLYQAKRFKVRVGDRNTEQEEGGEAVHEVEVVIKHNRFTKETYDFDIAVLRLKTPITFRMNVAPACLPERDWAESTLMTQKTGIVSGFGRTHEKGRQSTRLKMLEVPYVDRNSCKLSSSFIITQNMFCAGYDTKQEDACQGDSGGPHVTRFKDTYFVTGIVSWGEGCARKGKYGIYTKVTAFLKWIDRSMKTRGLPKAKSHAPEVITSSPLK

Sequence length: 488 aa.


In [4]:
# List of pdbis
num_pdb_ids = 144
pdbids_file = F'../data/{prot_name.upper()}_pdb_IDs_{uniprot_id}.csv'

if os.path.isfile(pdbids_file):
    pdbids_df = pd.read_csv(pdbids_file)
else:
    pdbids_df = pdb_ids_from_uniprot(uniprot_id)
    pdbids_df.to_csv(pdbids_file)

# Get a list
pdbids_list = pdbids_df['pdb_id'].tolist()
pdbids_chains = pdbids_df['chain'].tolist()
print(F'{len(pdbids_list)} pdb ids were found.')

136 pdb ids were found.


In [5]:
pdbids_list.sort()

In [6]:
from modules.get_seq_data import get_data

In [7]:
seq_prot

'MGRPLHLVLLSASLAGLLLLGESLFIRREQANNILARVTRANSFLEEMKKGHLERECMEETCSYEEAREVFEDSDKTNEFWNKYKDGDQCETSPCQNQGKCKDGLGEYTCTCLEGFEGKNCELFTRKLCSLDNGDCDQFCHEEQNSVVCSCARGYTLADNGKACIPTGPYPCGKQTLERRKRSVAQATSSSGEAPDSITWKPYDAADLDPTENPFDLLDFNQTQPERGDNNLTRIVGGQECKDGECPWQALLINEENEGFCGGTILSEFYILTAAHCLYQAKRFKVRVGDRNTEQEEGGEAVHEVEVVIKHNRFTKETYDFDIAVLRLKTPITFRMNVAPACLPERDWAESTLMTQKTGIVSGFGRTHEKGRQSTRLKMLEVPYVDRNSCKLSSSFIITQNMFCAGYDTKQEDACQGDSGGPHVTRFKDTYFVTGIVSWGEGCARKGKYGIYTKVTAFLKWIDRSMKTRGLPKAKSHAPEVITSSPLK'

In [16]:
prot_crys_mtd = {} # Diccionario temporal para posteriormente guardar en un archivo de texto
import time
start = time.time()

struct_DIR = F'../../FILES/CRYSTALS/PROT_{prot_name.upper()}_CHAINS/'

# ¿Está ordenada alfanuméricamente?
print("¿Está la lista ordenada?: ", pdbids_list[0:5])


# Primero se crea el diccionario
for pdb, chain in zip(pdbids_list, pdbids_chains):
    try:
        '''Para obtener los metadatos de todos los cristales'''
        descrip_pdb = pypdb.describe_pdb(pdb)
        cristal = pypdb.get_entity_info(pdb)
        ligando = pypdb.get_ligands(pdb)['ligandInfo']
        # Ommit the sequence
        # secuencia = pdbids[pdb]
        seq_alg, covertura, gaps = get_data(pdb, seq_prot,
                                            struct_dir = struct_DIR, 
                                            tail_pdb = '_A.pdb')
        #print(seq_alg)
        if ligando != None:
            ligando = ligando['ligand']
            if isinstance(ligando, dict): 
                # Si me devuelve un dict significa que sólo hay un lig
                ligando = [ligando] # Anida el dict a una lista, tal como
                # ocurre cuando hay más de un ligando
            num_ligs = len(ligando)
            # Lista del nombre de los ligandos
            ligs_names = [ligando[mol]['@chemicalID'] for mol in range(num_ligs)]
        else:
            ligs_names = []
            num_ligs = 0
        prot_crys_mtd.update({pdb : {'describe_pdb': descrip_pdb, 'pdb_info': cristal, 'ligs': ligando, 
                                     'num_ligs': num_ligs, 'name_ligs': ligs_names,'seq': '', 'chain_id': chain,
                                     'seq_alg': seq_alg, 'covertura': covertura, 'gaps': gaps}})
    except Exception as e:
        print(str(e) + ": Error con " + pdb)
# Finalmente el diccionario generado posee un diccionario para cada clave pdb,
# que, a su vez, posee cinco entradas: pdb_info, ligs, ligs_names, num_ligs, seq
    
end = time.time()
print(end - start)
print(len(prot_crys_mtd))

¿Está la lista ordenada?:  ['1c5m', '1ezq', '1f0r', '1f0s', '1fax']


@> 3781 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 1852 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1852 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1852 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 2295 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1852 atoms and 1 coordinate set(s) were parsed in 0.03s.
@> 1856 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1856 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1863 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1864 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1864 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1864 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1864 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1864 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1864 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1864 atoms and 1 coordinate set(s) were parsed in 0.02s.
@> 1864 atoms and 1 coordinate set(s) we

126.86256313323975
136


In [17]:
## GUARDAR
# Versión JSON
import json
n_pdb_id = len(pdbids_list)
json.dump(prot_crys_mtd, open(F"../data/MTDATA_{prot_name.upper()}_{str(n_pdb_id)}_crys.json",'w'))

## LEER el archivo json creado
prot_crys_mtd = json.load(open(F"../data/MTDATA_{prot_name.upper()}_{len(pdbids_list)}_crys.json", 'r'))

In [19]:
# PDB_ID, Título, #Entidades, #Cadenas, Cadena, Covertura, #Gaps, ListaGaps, #residuos, Resulucion, método, #Ligs, IDs Ligs, 
# Nombres Ligs, Lig más grande, Peso Lig más grande
def get_data_rows(pdb_entry):
    x = pdb_entry
    d = x['describe_pdb']
    s = x['seq']
    l = x['ligs']

    dic = {"PDB_ID": d['structureId'].lower(),
          "Title": d['title'].lower(),
          "Entities": d['nr_entities'],
          "ChainID": x['chain_id'],
          "Identity": '-',
          "Coverage": x['covertura'],
          "NumGaps": x['gaps']['num_gaps'],
          "GapLen": x['gaps']['gap_lengths'],
          "GapPos": x['gaps']['gap_list'],
          #"NumResidues": ,
          "Resolution": d['resolution'],
          "Date": d['release_date'],
          "NumLigs": x['num_ligs'], 
          "NameLigs": x['name_ligs'],
          }
    return(pd.Series(dic))

def get_mtd_table(dic):
    df = pd.DataFrame()
    for pdb_id in dic.keys():
        row = get_data_rows(dic[pdb_id])
        df = df.append(row, ignore_index=True)
    return(df)

In [20]:
# Se genera el dataframe
df_prot = get_mtd_table(prot_crys_mtd).sort_values("PDB_ID")
print("La tabla de datos tiene registradas", str(len(df_prot)), "estructuras.")

# Removemos las estructuras que no servirán para continuar el análisis
# Estas sestructuras son identificadas en el paso de modelación con Modeller
# con lo cual la lista debe ser actualizada si se identifican estructuras que no puedan ser modeladas
droped_pdb_ids = []

df_prot = df_prot.loc[ ~df_prot["PDB_ID"].isin(droped_pdb_ids) ]
print("La NUEVA tabla de datos tiene registradas", str(len(df_prot)), "estructuras.")

# Reordenando la tabla
df_prot = df_prot.reset_index()

# Se reordenan las columnas
df_prot = df_prot[['PDB_ID', 'Title','Date', 'Entities', 'ChainID','Resolution', 
         'Identity', 'Coverage', 'NumGaps', 'GapLen', 'GapPos',
         'NumLigs', 'NameLigs']]

La tabla de datos tiene registradas 136 estructuras.
La NUEVA tabla de datos tiene registradas 136 estructuras.


In [21]:
df_prot

,PDB_ID,Title,Date,Entities,ChainID,Resolution,Identity,Coverage,NumGaps,GapLen,GapPos,NumLigs,NameLigs
0,1c5m,structural basis for selectivity of a small mo...,2000-12-22,2,D,1.95,-,49.385246,2.0,"[235, 14]","[[1, 235], [475, 488]]",0.0,[]
1,1ezq,crystal structure of human coagulation factor ...,2000-09-20,2,A,2.20,-,47.950820,2.0,"[235, 21]","[[1, 235], [469, 489]]",2.0,"[CA, RPR]"
2,1f0r,crystal structure of human coagulation factor ...,2000-09-20,2,A,2.10,-,47.950820,2.0,"[235, 21]","[[1, 235], [469, 489]]",2.0,"[815, CA]"
3,1f0s,crystal structure of human coagulation factor ...,2000-09-20,2,A,2.10,-,47.950820,2.0,"[235, 21]","[[1, 235], [469, 489]]",2.0,"[CA, PR2]"
4,1fax,coagulation factor xa inhibitor complex,1997-10-29,2,A,3.00,-,48.155738,2.0,"[235, 20]","[[1, 235], [470, 489]]",2.0,"[CA, DX9]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,4zh8,factor xa complex with gtc000006,2016-01-13,2,A,1.80,-,47.950820,2.0,"[235, 21]","[[1, 235], [469, 489]]",2.0,"[4O4, CA]"
132,4zha,factor xa complex with gtc000102,2016-01-13,2,A,1.86,-,47.950820,2.0,"[235, 21]","[[1, 235], [469, 489]]",3.0,"[4O5, CA, MG]"
133,5k0h,human factor xa in complex with synthetic inhi...,2016-07-06,2,A,2.20,-,47.950820,2.0,"[235, 21]","[[1, 235], [469, 489]]",2.0,"[6PK, CA]"
134,5voe,desgla-xas195a bound to aptamer 11f7t,2018-06-20,3,H,2.00,-,47.336066,3.0,"[235, 3, 22]","[[1, 235], [370, 372], [467, 488]]",5.0,"[CA, CFL, MG, NA, UFT]"


In [22]:
n_pdb_id_to_use = len(df_prot)
df_prot.to_json(F"../data/TABLA_MTDATA_{prot_name.upper()}_{str(n_pdb_id_to_use)}_crys.json")